In [3]:
import numpy as np
from netCDF4 import Dataset
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.neural_network import MLPClassifier

In [4]:
f1 = Dataset('../multi_eof.nc', 'r')
f2 = Dataset('../gpcc.r.1x1.classified.nc', 'r')
r = f2.variables['r']
x0 = f1.variables['eof_ts']
x = np.transpose(x0)

In [3]:
lat = f2.variables['lat'][:]
lon = f2.variables['lon'][:]
lat_bnds, lon_bnds = [-18, -2], [287, 313]
lat_inds = np.where((lat > lat_bnds[0]) & (lat < lat_bnds[1]))[0]
lon_inds = np.where((lon > lon_bnds[0]) & (lon < lon_bnds[1]))[0]
#y = rain.variables['r'][:,:,np.min(lat_inds):np.max(lat_inds),np.min(lon_inds):np.max(lon_inds)]
y = f2.variables['r'][:,:,lat_inds,lon_inds]

In [4]:
nyear = len(f2.variables['year'])
nmonth = len(f2.variables['month'])
nlat = len(lat_inds)
nlon = len(lon_inds) 

# This is for CNN to find out the best alpha

In [5]:
def sigmoid(z):
    return np.divide(1.0,(1.0 + np.exp(-z)))
    
def compute_cost(Y, predictY):
    y = np.zeros(len(predictY))
    y[Y-np.min(Y)] = 1
    Jc = np.sum(np.dot(-y,np.log(sigmoid(predictY)))-np.dot((1-y),np.log(1.-sigmoid(predictY))))
    return Jc

In [9]:
n_alphas = 200
alphas = np.logspace(-8, 8, n_alphas)

k_list = [i for i in range(0,nyear,3)]
k_list[len(k_list)-1] = nyear # make last year

alpha_map = np.zeros((nmonth,nlat,nlon))
for imonth in range(nmonth): # each month has its own model
    for ilat in range(nlat): # each location has its own model
        for ilon in range(nlon):
            tRSS = []
            
            for ia in range(len(alphas)):
                val_RSS = 0.
                
                for ik in range(len(k_list)-1):
                    val_set = [i for i in range(k_list[ik],k_list[ik+1])]
                    train_set = [i for j, i in enumerate(range(nyear)) if j not in val_set]
                    
                    reg = MLPClassifier(solver='lbgfs', alpha=alphas[ia],hidden_layer_sizes=(5,), random_state=1)
                    reg.fit(x[train_set,:],y[train_set,imonth,ilat,ilon])
                    predictY = reg.predict_proba(x[val_set,:])
                    
                    for iv in range(len(val_set)):
                        val_RSS = val_RSS + compute_cost(y[val_set[iv],imonth,ilat,ilon],predictY[iv,:])
                    
                    del val_set,train_set,predictY,reg
                
                tRSS.append(val_RSS)
                del val_RSS
                
            alpha_map[imonth,ilat,ilon] = alphas[np.argmin(tRSS)]
            del tRSS
print("completed!")

completed!


In [10]:
predictY = np.zeros((nyear,nmonth,nlat,nlon))
                               
for imonth in range(nmonth):
    for ilat in range(nlat):
        for ilon in range(nlon):
            reg = LogisticRegression (C = alpha_map[imonth,ilat,ilon])
            reg.fit(x[:,:],y[:,imonth,ilat,ilon])
            predictY[:,imonth,ilat,ilon] = reg.predict(x)
            del reg
print("completed!")

completed!


# with regulization

In [11]:
predictY = np.zeros((nyear,nmonth,nlat,nlon))
accuracy_score = np.zeros((nmonth,nlat,nlon))

for ilat in range(nlat):
    for ilon in range(nlon):
        for imonth in range(nmonth):
            reg = MLPClassifier(solver='lbgfs', alpha=alpha_map[imonth,ilat,ilon],hidden_layer_sizes=(5,), random_state=1)
            reg.fit(x[:,:],y[:,imonth,ilat,ilon])
            predictY[:,imonth,ilat,ilon] = reg.predict(x)
            #print(reg.predict_proba(x).shape)
            del reg
            accuracy_score[imonth,ilat,ilon] = metrics.accuracy_score(y[:,imonth,ilat,ilon], predictY[:,imonth,ilat,ilon])

In [12]:
pr = Dataset('predict.r.CNN.regulization.nc', 'w',format='NETCDF3_64BIT')
pr.description = 'predicted rainfall anomalies using simple CNN method'

pr.createDimension('year', nyear)
pr.createDimension('month', nmonth)
pr.createDimension('lat', nlat)
pr.createDimension('lon', nlon)

fyear = pr.createVariable('year', 'f', ('year',))
fmonth = pr.createVariable('month', 'f', ('month',))
flat = pr.createVariable('lat', 'f', ('lat',))
flon = pr.createVariable('lon', 'f', ('lon',))
newr = pr.createVariable('r', 'f4', ('year', 'month','lat','lon'),fill_value=-999)
acc = pr.createVariable('acc', 'f4', ('month','lat','lon'),fill_value=-999)

fyear[:] = f2.variables['year']
fmonth[:] = f2.variables['month']
flat[:] = f2.variables['lat'][lat_inds]
flon[:] = f2.variables['lon'][lon_inds]
newr[:,:,:,:] = predictY[:,:,:,:]
acc[:,:,:] = accuracy_score[:,:,:]
flat.units = "degrees_north"
flat.long_name = "Latitude"
flon.units = "degrees_east"
flon.long_name = "Longitude"
newr.long_name = 'predicted rainfall anomalies using simple logistic regression'
pr.close()